In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
import pickle as pkl
from hierataxo.util import (
    OrderManager)
from hierataxo import (
    OrderManager,ConcatProteinDataset,
    HierarchicalLossNetwork,
    cal_accuracy,set_seed)
from hierataxo.dataset import ConcatProteinDataModule
from hierataxo.model import HierarESM
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, DistributedSampler,random_split
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint,LearningRateMonitor

from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger

# L.Trainer()
max_length=500
max_domain=15
train_bs=2
infer_bs=30
seed=42
seed_everything(seed, workers=True)
order_manager=OrderManager(pkl.load(open('taxo_data/hierarchy_order.pkl','rb'))['Riboviria'],
                        level_names=['Kingdom','Phylum','Class','Order'])
model=HierarESM(order_manager,max_length=max_length,max_domain=max_domain,
    optimizer_kwargs={'backbone_lr':1e-4,'head_lr':1e-3,'weight_decay':0.01},
    scheduler_kwargs={'warmup_iter_1':20,'warmup_iter_2':30,'warmup_lr':1e-10})

datamodule=ConcatProteinDataModule(order_manager,'taxo_data/proseq_taxo_1.pkl',
    max_length=max_length,max_domain=max_domain,train_bs=train_bs,infer_bs=infer_bs)
# datamodule.setup('fit')

Seed set to 42


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [76]:
import os
from lightning.pytorch.callbacks import BasePredictionWriter
from typing import List,Dict
from itertools import chain
import pandas as pd
import torch
class PredictWriter(BasePredictionWriter):
    def __init__(self, outname:str):
        super().__init__(write_interval='epoch')
        self.outname = outname

    def write_on_epoch_end(self, trainer, pl_module, 
        predictions:List[Dict[str,List[str]|torch.Tensor]], batch_indices):
        o={}
        dtypes={k:type(v) for k,v in predictions[0].items()}
        for k,dt in dtypes:
            if dt is list:
                o[k]=sum([i[k] for i in predictions],start=[])
            elif dt is torch.Tensor:
                o[k]=torch.concat([i[k] for i in predictions]).tolist()
        pd.DataFrame(o).to_pickle(trainer.default_root_dir+'/'+self.outname)
        # TODO use hdf5 or lmdb to write on batch
        # torch.save(predictions, os.path.join(self.output_dir, "predictions.pt"))


In [77]:
from glob import glob
from pathlib import Path
default_root_dir='infer'
exp_dir='reload_ckpt'
for i in Path('tmp_poster_ana/used_models/').iterdir():
    seed=i.stem.split('-')[1]
    model.load_state_dict(torch.load(i,map_location='cpu'),strict=False)
    trainer = L.Trainer(
        default_root_dir=default_root_dir, 
        accelerator='gpu',
        strategy='auto',
        max_epochs=100,
        check_val_every_n_epoch=1,
        log_every_n_steps=1,
        precision='32',
        max_steps=0,
        callbacks=PredictWriter(f'{exp_dir}/seed-{seed}.pkl')
        # model.on_save_checkpoint
        # limit_predict_batches=4,
        # limit_train_batches=100,
        # limit_val_batches=30,
        # callbacks=[ModelCheckpoint(dirpath=default_root_dir+'/'+exp_dir,monitor='val_loss',
        #                 filename='checkpoint-{step:06d}-{val_loss:.2f}',save_top_k=2),
        #            LearningRateMonitor('epoch',log_weight_decay=True)],
        # logger=TensorBoardLogger(save_dir=default_root_dir,name=exp_dir)
        )
    trainer.predict(model,datamodule)
    trainer.save_checkpoint(f'{exp_dir}/seed-{seed}-last.ckpt')

/tmp/ipykernel_35453/2596810752.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(i,map_location='cpu'),strict=False)
GPU available: True 

Predicting DataLoader 0:  23%|██▎       | 52/222 [00:41<02:16,  1.25it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.34 GiB. GPU 0 has a total capacity of 14.57 GiB of which 2.28 GiB is free. Including non-PyTorch memory, this process has 12.28 GiB memory in use. Of the allocated memory 6.42 GiB is allocated by PyTorch, and 5.74 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [61]:
o=trainer.predict(model,datamodule)
# trainer.save_checkpoint('seed-7-last.ckpt')

/home/taxo_playground/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00,  5.73it/s]


In [68]:
predictions=o
o_={}
dtypes={k:type(v) for k,v in predictions[0].items()}
for k,dt in dtypes.items():
    if dt is list:
        o_[k]=sum([i[k] for i in predictions],start=[])
    elif dt is torch.Tensor:
        o_[k]=torch.concat([i[k] for i in predictions],).tolist()
p

In [63]:
o_['predict_labels']

[['Orthornavirae', 'Lenarviricota', 'Miaviricetes', 'Ourlivirales'],
 ['Orthornavirae', 'Kitrinoviricota', 'Alsuviricetes', 'Tymovirales'],
 ['Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Monjiviricetes', 'Mononegavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Monjiviricetes', 'Mononegavirales'],
 ['Orthornavirae', 'Pisuviricota', 'Stelpaviricetes', 'Stellavirales'],
 ['Orthornavirae', 'Pisuviricota', 'Stelpaviricetes', 'Stellavirales'],
 ['Orthornavirae', 'Pisuviricota', 'Stelpaviricetes', 'Stellavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Monjiviricetes', 'Mononegavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Monjiviricetes', 'Mononegavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Ellioviricetes', 'Bunyavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Ellioviricetes', 'Bunyavirales']]

In [69]:
import pandas as pd
pd.DataFrame(o_)

,name,seq,taxo_label,domain_label,taxo,sentence_mask,input_ids,attention_mask,Kingdom_predict,Phylum_predict,Class_predict,Order_predict,predict_labels
0,6PavOLV7||MN532594,[RKLKPSQLLQVETGGKYRIVSKSDLGMNSLRPLHSAIYNHLSRFS...,"[Orthornavirae, Lenarviricota, Miaviricetes, O...","[Mitovir_RNA_pol, , , , , , , , , , , , , , ]","[1, 3, 11, 14]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 10, 15, 4, 15, 14, 8, 16, 4, 4, 16, 7, 9,...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[-17.12173080444336, 7.148506164550781, -11.82...","[-16.606761932373047, -12.017447471618652, -13...","[-18.014789581298828, -18.111919403076172, -16...","[-16.4014949798584, -20.295434951782227, -19.6...","[Orthornavirae, Lenarviricota, Miaviricetes, O..."
1,AAV1||KF421905,[HYNMPIEAATKLESFGIITNPYSIALHSHAAAKVCENKLLDTVGH...,"[Orthornavirae, Kitrinoviricota, Alsuviricetes...","[Vmethyltransf, FTO_NTD, Viral_helicase1, RdRP...","[1, 2, 4, 6]","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 21, 19, 17, 20, 14, 12, 9, 5, 5, 11, 15, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[-15.91338062286377, 4.330357074737549, -10.42...","[-17.33768653869629, -15.704703330993652, -1.6...","[-10.867238998413086, -12.22330093383789, -11....","[-13.037378311157227, -16.32259750366211, -12....","[Orthornavirae, Kitrinoviricota, Alsuviricetes..."
2,AAbV||GBBW01007738,[GLCSGVLVGENLVLTAAHCLVQPAQVLRFCLGDDCYQILSYEVVD...,"[Orthornavirae, Pisuviricota, Pisoniviricetes,...","[DUF316, DUF2075, RdRP_2, CoV_RPol_N, LAP1_C, ...","[1, 5, 19, 23]","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 6, 4, 23, 8, 6, 7, 4, 7, 6, 9, 17, 4, 7, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[-15.944304466247559, 4.202620983123779, -11.1...","[-12.274317741394043, -12.27157974243164, -9.0...","[-16.270158767700195, -14.265697479248047, -14...","[-11.492676734924316, -11.494980812072754, -10...","[Orthornavirae, Pisuviricota, Pisoniviricetes,..."
3,AAnV1|0|BK059208,[NGSFSFRLGKNLHALEVMGTNKLYVGNNNSSLLLLDTLGQRLCLE...,"[Orthornavirae, Negarnaviricota, Monjiviricete...","[Mononeg_RNA_pol, Mononeg_mRNAcap, Methyltrans...","[1, 4, 14, 18]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 17, 6, 8, 18, 8, 18, 10, 4, 6, 15, 17, 4,...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[-12.73079776763916, 12.338814735412598, -7.33...","[-16.030885696411133, -10.683784484863281, -13...","[-17.23851776123047, -15.97236156463623, -14.7...","[-17.72771644592285, -16.68787384033203, -12.2...","[Orthornavirae, Negarnaviricota, Monjiviricete..."
4,AAnV1|1|BK059209,[WPDSRMHGKTGIRLIYLSTANRLRIGKRVINDLHTGISDKTIGGI...,"[Orthornavirae, Negarnaviricota, Monjiviricete...","[Rhabdo_ncap_2, , , , , , , , , , , , , , ]","[1, 4, 14, 18]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 22, 14, 13, 8, 10, 20, 21, 6, 15, 11, 6, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[-14.70251178741455, 8.073880195617676, -3.283...","[-11.194315910339355, -7.476253509521484, -8.6...","[-7.324009895324707, -4.696815013885498, -3.05...","[-10.98635196685791, -6.811470985412598, -3.93...","[Orthornavirae, Negarnaviricota, Monjiviricete..."
5,AAstV1||Y15936,[SIVVVEGQGGSGVGWRFMNSIFTAGHVVQGSKFVTIKSESTQVKV...,"[Orthornavirae, Pisuviricota, Stelpaviricetes,...","[Peptidase_C62, Astro_VPg, RdRP_1, Astro_capsi...","[1, 5, 20, 27]","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 8, 12, 7, 7, 7, 9, 6, 16, 6, 6, 8, 6, 7, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[-13.615700721740723, 2.0321462154388428, -5.1...","[-10.854243278503418, -9.116840362548828, -8.3...","[-7.915549278259277, -13.054832458496094, -11....","[-10.215134620666504, -12.144798278808594, -12...","[Orthornavirae, Pisuviricota, Stelpaviricetes,..."
6,AAstV2||AB033998,[FPSKADNIVKIEVDVDGGSAGVGFRLGNYIYTAGHVVGEAKIAKI...,"[Orthornavirae, Pisuviricota, Stelpaviricetes,...","[Peptidase_S32, Astro_VPg, RdRP_2, Astro_capsi...","[1, 5, 20, 27]","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[0, 18, 

In [64]:
o_['taxo_label']

[['Orthornavirae', 'Lenarviricota', 'Miaviricetes', 'Ourlivirales'],
 ['Orthornavirae', 'Kitrinoviricota', 'Alsuviricetes', 'Tymovirales'],
 ['Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Monjiviricetes', 'Mononegavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Monjiviricetes', 'Mononegavirales'],
 ['Orthornavirae', 'Pisuviricota', 'Stelpaviricetes', 'Stellavirales'],
 ['Orthornavirae', 'Pisuviricota', 'Stelpaviricetes', 'Stellavirales'],
 ['Orthornavirae', 'Pisuviricota', 'Stelpaviricetes', 'Stellavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Monjiviricetes', 'Mononegavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Monjiviricetes', 'Mononegavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Ellioviricetes', 'Bunyavirales'],
 ['Orthornavirae', 'Negarnaviricota', 'Ellioviricetes', 'Bunyavirales']]

In [52]:
0.98**50

0.36416968008711675

In [11]:
for i in o[0]['predictions']:
    print(model.order_manager.idx_to_order(torch.argmax(i, dim=1).tolist()))

tensor([1, 1, 1])
tensor([3, 2, 5])
tensor([11,  4, 19])
tensor([14,  6, 23])


In [36]:
_=model.order_manager.idx_to_order(o[0]['predictions'],need_argmax=True)
# [list(row) for row in zip(*_)]
_

[['Orthornavirae', 'Lenarviricota', 'Miaviricetes', 'Ourlivirales'],
 ['Orthornavirae', 'Kitrinoviricota', 'Alsuviricetes', 'Tymovirales'],
 ['Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales']]

In [45]:
dtypes={k:type(v) for k,v in o[0].items()}

In [50]:
dtypes['attention_mask'] is torch.Tensor

True

In [44]:
{k:type(v) for k,v in o[0].items()}

{'name': list,
 'seq': list,
 'taxo_label': list,
 'domain_label': list,
 'taxo': torch.Tensor,
 'sentence_mask': torch.Tensor,
 'input_ids': torch.Tensor,
 'attention_mask': torch.Tensor,
 'predictions': list,
 'acc_Kingdom': torch.Tensor,
 'acc_Phylum': torch.Tensor,
 'acc_Class': torch.Tensor,
 'acc_Order': torch.Tensor}

In [67]:
{k:len(v) for k,v in o_.items() if not isinstance(v,int) and not 'acc' in k}

{'name': 12,
 'seq': 12,
 'taxo_label': 12,
 'domain_label': 12,
 'taxo': 12,
 'sentence_mask': 12,
 'input_ids': 12,
 'attention_mask': 12,
 'Kingdom_predict': 12,
 'Phylum_predict': 12,
 'Class_predict': 12,
 'Order_predict': 12,
 'predict_labels': 12}

In [24]:
o_=[]
idx_list=o[0]['predictions']
for i,level in enumerate(model.order_manager.levels):
    idx_l=torch.argmax(idx_list[i],dim=1).tolist() if isinstance(
        idx_list[i],torch.Tensor) else idx_list[i]
    o_.append([level[j] for j in idx_l])

In [25]:
o_

[['Orthornavirae', 'Orthornavirae', 'Orthornavirae'],
 ['Lenarviricota', 'Kitrinoviricota', 'Pisuviricota'],
 ['Miaviricetes', 'Alsuviricetes', 'Pisoniviricetes'],
 ['Ourlivirales', 'Tymovirales', 'Nidovirales']]

In [22]:
torch.argmax(idx_list[i],dim=1)

tensor([14,  6, 23])